<a href="https://colab.research.google.com/github/Bhavana0929/Amenity-Detection/blob/main/explorer_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
# These are the subset of classes Airbnb are most concerned with
subset = ["Toilet",
          "Swimming_pool",
          "Bed",
          "Billiard_table",
          "Sink",
          "Fountain",
          "Oven",
          "Ceiling_fan",
          "Television",
          "Microwave_oven",
          "Gas_stove",
          "Refrigerator",
          "Kitchen_&_dining_room_table",
          "Washing_machine",
          "Bathtub",
          "Stairs",
          "Fireplace",
          "Pillow",
          "Mirror",
          "Shower",
          "Couch",
          "Countertop",
          "Coffeemaker",
          "Dishwasher",
          "Sofa_bed",
          "Tree_house",
          "Towel",
          "Porch",
          "Wine_rack",
          "Jacuzzi"]

In [ ]:
len(subset)

30

# **Start exploring the class names in Open Images**

Downloaded the class descriptions from Open Images: !wget https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv

This file contains all of the codenames for the classes which have bounding box labels in Open Images.

In [ ]:
!wget https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv

--2025-01-17 17:34:42--  https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.62.207, 142.251.163.207, 142.251.167.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.62.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11255 (11K) [text/csv]
Saving to: ‘class-descriptions-boxable.csv’

class-descriptions- 100%[===================>]  10.99K  --.-KB/s    in 0s      

2025-01-17 17:34:42 (50.9 MB/s) - ‘class-descriptions-boxable.csv’ saved [11255/11255]



In [ ]:
# All the classes in Open Images
classes = pd.read_csv("class-descriptions-boxable.csv",names=["ID","Names"])
classes

,ID,Names
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football
...,...,...
596,/m/0qmmr,Wheelchair
597,/m/0wdt60w,Rugby ball
598,/m/0xfy,Armadillo
599,/m/0xzly,Maracas


In [ ]:
# Let's get a subset or at least all the columns which match
classes["match"] = classes["Names"].isin(subset)
classes

,ID,Names,match
0,/m/011k07,Tortoise,False
1,/m/011q46kg,Container,False
2,/m/012074,Magpie,False
3,/m/0120dh,Sea turtle,False
4,/m/01226z,Football,False
...,...,...,...
596,/m/0qmmr,Wheelchair,False
597,/m/0wdt60w,Rugby ball,False
598,/m/0xfy,Armadillo,False
599,/m/0xzly,Maracas,False


In [ ]:
classes.match.value_counts()

,count
match,
False,581
True,20


In [ ]:
# Where do they match up?
matches = classes[classes["match"]==True]["Names"].tolist()
matches

['Sink',
 'Towel',
 'Stairs',
 'Fountain',
 'Oven',
 'Couch',
 'Shower',
 'Pillow',
 'Bathtub',
 'Bed',
 'Fireplace',
 'Refrigerator',
 'Porch',
 'Mirror',
 'Jacuzzi',
 'Television',
 'Coffeemaker',
 'Toilet',
 'Countertop',
 'Dishwasher']

In [ ]:
# Where are they different?
missing_classes = list(set(subset)-set(matches))
missing_classes

['Ceiling_fan',
 'Gas_stove',
 'Sofa_bed',
 'Swimming_pool',
 'Wine_rack',
 'Kitchen_&_dining_room_table',
 'Tree_house',
 'Washing_machine',
 'Microwave_oven',
 'Billiard_table']

In [ ]:
# Are there similar versions of these classes in the descriptions I could use?
classes[classes["Names"].str.contains("pool")]

,ID,Names,match
444,/m/0b_rs,Swimming pool,False


In [ ]:
classes[classes["Names"].str.contains("stove")]

,ID,Names,match
197,/m/02wv84t,Gas stove,False
270,/m/04169hn,Wood-burning stove,False


In [ ]:
classes[classes["Names"].str.contains("stove")]["Names"].tolist()

['Gas stove', 'Wood-burning stove']

In [ ]:
# Get the individual words from each string of missing classes
strings = [x.split("_") for x in missing_classes]
strings = [item for sublist in strings for item in sublist]
strings

['Ceiling',
 'fan',
 'Gas',
 'stove',
 'Sofa',
 'bed',
 'Swimming',
 'pool',
 'Wine',
 'rack',
 'Kitchen',
 '&',
 'dining',
 'room',
 'table',
 'Tree',
 'house',
 'Washing',
 'machine',
 'Microwave',
 'oven',
 'Billiard',
 'table']

In [ ]:
# Now find if any of the strings match up
more_matches = []
for string in strings:
  more_matches.append(classes[classes["Names"].str.contains(string)]["Names"].tolist())
more_matches = list(set([item for sublist in more_matches for item in sublist]))
more_matches

['Wine',
 'Tree house',
 'Bathroom accessory',
 'Kitchen appliance',
 'Lighthouse',
 'Swimming pool',
 'Kitchen & dining room table',
 'Washing machine',
 'Microwave oven',
 'Sofa bed',
 'Tennis racket',
 'Mushroom',
 'Vegetable',
 'Infant bed',
 'Kitchenware',
 'Wine glass',
 'Bathroom cabinet',
 'Billiard table',
 'Tree',
 'Coffee table',
 'Table tennis racket',
 'Spice rack',
 'Wood-burning stove',
 'Dog bed',
 'Sewing machine',
 'Wine rack',
 'Kitchen utensil',
 'Gas stove',
 'Ceiling fan',
 'Kitchen knife',
 'Mechanical fan']

In [ ]:
# Take out the underscore
missing_classes_no_space = [x.replace("_"," ") for x in missing_classes]
missing_classes_no_space

['Ceiling fan',
 'Gas stove',
 'Sofa bed',
 'Swimming pool',
 'Wine rack',
 'Kitchen & dining room table',
 'Tree house',
 'Washing machine',
 'Microwave oven',
 'Billiard table']

In [ ]:
# Find the actual missing classes
actual_missing_classes = list(set(missing_classes_no_space)-set(more_matches))
actual_missing_classes

[]

Turns out there aren't any missing classes from the Open Images set! The only difference here is the naming convention. Airbnb used underscores "_" in their class names. This is a simple fix we can implement later.

Let's remove the underscores from our subset list and play with that to start downloading classes.

In [ ]:
subset_no_underscore = [x.replace("_"," ") for x in subset]
subset_no_underscore

['Toilet',
 'Swimming pool',
 'Bed',
 'Billiard table',
 'Sink',
 'Fountain',
 'Oven',
 'Ceiling fan',
 'Television',
 'Microwave oven',
 'Gas stove',
 'Refrigerator',
 'Kitchen & dining room table',
 'Washing machine',
 'Bathtub',
 'Stairs',
 'Fireplace',
 'Pillow',
 'Mirror',
 'Shower',
 'Couch',
 'Countertop',
 'Coffeemaker',
 'Dishwasher',
 'Sofa bed',
 'Tree house',
 'Towel',
 'Porch',
 'Wine rack',
 'Jacuzzi']

Okay we'll start with a small class (small as in, there are likely not many examples), let's use Jacuzzi first.

Get all the files we need from Open Images (labels, annotations, descriptions, etc)

In [ ]:
!wget https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv

!wget https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv

!wget https://storage.googleapis.com/openimages/2018_04/validation/validation-annotations-bbox.csv

!wget https://storage.googleapis.com/openimages/2018_04/test/test-annotations-bbox.csv

--2025-01-17 18:06:05--  https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.122.207, 172.253.63.207, 142.250.31.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.122.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11255 (11K) [text/csv]
Saving to: ‘class-descriptions-boxable.csv.1’

class-descriptions- 100%[===================>]  10.99K  --.-KB/s    in 0s      

2025-01-17 18:06:05 (73.9 MB/s) - ‘class-descriptions-boxable.csv.1’ saved [11255/11255]

--2025-01-17 18:06:05--  https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.122.207, 172.253.63.207, 142.250.31.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.122.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1194033454 (1

In [ ]:
!pip install awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: docutils
    Found existing installation: docutils 0.21.2
    Uninstalling docutils-0.21.2:
      Successfully uninstalled docutils-0.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.1.3 requires docutils<0.22,>=0.20, but you have docutils 0.16 which is incompatible.


In [ ]:
!python3 downloadOI.py --classes 'Toilet,Bathtub' --mode validation

CPU count: 2
usage: downloadOI.py [-h] --dataset DATASET --classes CLASSES [--nthreads NTHREADS]
                     [--occluded OCCLUDED] [--truncated TRUNCATED] [--groupOf GROUPOF]
                     [--depiction DEPICTION] [--inside INSIDE]
downloadOI.py: error: the following arguments are required: --dataset


In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
!wget "https://raw.githubusercontent.com/mrdbourke/airbnb-amenity-detection/refs/heads/master/downloadOI.py"

--2025-01-17 18:14:56--  https://raw.githubusercontent.com/mrdbourke/airbnb-amenity-detection/refs/heads/master/downloadOI.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4269 (4.2K) [text/plain]
Saving to: ‘downloadOI.py’

downloadOI.py       100%[===================>]   4.17K  --.-KB/s    in 0s      

2025-01-17 18:14:56 (23.1 MB/s) - ‘downloadOI.py’ saved [4269/4269]



In [ ]:
!python3 downloadOI.py --classes 'Jacuzzi' --dataset train

CPU count: 2
grep /m/065h6l ./train-annotations-bbox.csv
Downloading: 102 images | Num classes: 1 | Dataset: train
100% 102/102 [01:33<00:00,  1.09it/s]
